In [1]:
import gymnasium as gym
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm import tqdm

/opt/homebrew/Caskroom/miniconda/base/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# print gym environment information
env = gym.make(
    "LunarLander-v2",
    continuous = False,
    gravity = -10.0,
    enable_wind = False,
    wind_power = 15.0,
    turbulence_power = 1.5,
    render_mode="rgb_array"
)
print('observation space:', env.observation_space)
print('action space:', env.action_space)


AttributeError: module '_Box2D' has no attribute 'RAND_LIMIT_swigconstant'

## Planning:
The goal is to use DQN to solve the lunar lander problem.

Currently I am unclear on the following:
Exactly how to set up the network to receive inputs and outputs from the system
What kind of loss function should be used in this setup to make sure that it is learning correctly.
How to account for the various kinds of things can be observed in the observation space.

## approach
A good first step would be to try and experiment with basic approaches messing around with the system to get a grasp of it.
Then I'd like to experiment with a basic neural network feeding in maybe 2 of the parameters.


In [1]:
def create_animation(images):
    fig = plt.figure()
    ims = []

    for image in images:
        im = plt.imshow(image, animated=True);
        ims.append([im])
    plt.close()

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000);

    return ani

In [2]:
for i in range(10):
    # reset environment
    state = env.reset()
    done = False
    images = []
    while not done:
        # render environment
        image = env.render()
        images.append(image)
        # sample random action
        action = env.action_space.sample()
        # take action
        state, reward, done, info = env.step(action)
    ani = create_animation(images)
    HTML(ani.to_html5_video())

NameError: name 'env' is not defined